In [ ]:
import pandas as pd

# Load the new dataset
data = pd.read_csv('players_22.csv', low_memory=False)

# Define the feature subset
feature_subset = [
    'potential', 'value_eur', 'wage_eur', 'age', 'international_reputation', 'shooting', 
    'passing', 'dribbling', 'physic', 'attacking_short_passing', 'skill_curve', 
    'skill_long_passing', 'skill_ball_control', 'movement_reactions', 'power_shot_power', 
    'power_long_shots', 'mentality_vision', 'mentality_composure', 'ls', 'st', 'rs', 
    'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 
    'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'rb'
]

# Calculate the default values (mean in this case)
default_values = data[feature_subset].mean().to_dict()

print(default_values)


In [ ]:
from flask import Flask, render_template, request
import pickle
import numpy as np
import pandas as pd

app = Flask(__name__)

# Load the trained XGBoost model
def load_model(model_path):
    with open(model_path, 'rb') as f:
        loaded_model = pickle.load(f)
    return loaded_model

# Path to your trained model
model_path = 'best_xgb_model.pkl'
model = load_model(model_path)

# Define the feature subset and load the dataset
feature_subset = [
    'potential', 'value_eur', 'wage_eur', 'age', 'international_reputation', 'shooting', 
    'passing', 'dribbling', 'physic', 'attacking_short_passing', 'skill_curve', 
    'skill_long_passing', 'skill_ball_control', 'movement_reactions', 'power_shot_power', 
    'power_long_shots', 'mentality_vision', 'mentality_composure', 'ls', 'st', 'rs', 
    'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 
    'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'rb'
]

# Calculate the default values
data = pd.read_csv('players_22.csv', low_memory=False)
default_values = data[feature_subset].mean().to_dict()

# Define the prediction route
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    # Retrieve and parse form data
    user_input = {
        'value_eur': float(request.form['value_eur']),
        'movement_reactions': float(request.form['movement_reactions']),
        'age': int(request.form['age']),
        'international_reputation': float(request.form['international_reputation']),
        # Add the rest of the user inputs
    }

    # Create a complete feature vector with default values
    input_features = np.array([default_values[feature] for feature in feature_subset])
    
    # Update the feature vector with user-provided values
    for feature, value in user_input.items():
        if feature in feature_subset:
            index = feature_subset.index(feature)
            input_features[index] = value

    # Reshape input_features to match model input shape
    input_features = input_features.reshape(1, -1)

    # Make prediction
    prediction = model.predict(input_features)[0]

    # Return the predicted result
    return f'Predicted Rating: {prediction:.2f}'

if __name__ == '__main__':
    app.run(debug=True)
